<a href="https://colab.research.google.com/github/panahiparham/mlclass_proj_f2021/blob/main/Experiments_QM9/Multitask_NN_QM9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

Dataset load

In [2]:
# path to QM9 features npy
x_path = '/content/drive/MyDrive/Data Science @ SBU/Applied Machine Learning/project/qm9_preprocessed/qm9_features.npy'


# path to QM9 labels npy
y_path = '/content/drive/MyDrive/Data Science @ SBU/Applied Machine Learning/project/qm9_preprocessed/qm9_labels.npy'


x = np.load(x_path)
y = np.load(y_path) 

In [3]:
print('x shape:', x.shape, 'dtype:', x.dtype)
print('y shape:', y.shape, 'dtype:', y.dtype)

x shape: (133885, 163) dtype: float64
y shape: (133885, 12) dtype: float64


Single task neural network

In [4]:
# task: gap (e_lumo - e_homo)
y_raw = y[:,4]

In [5]:
print(y_raw.min())
print(y_raw.max())

0.0246
0.6221


In [6]:
x_raw = x.copy()

In [17]:
# train test split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=0.20, random_state=42)

In [18]:
# model

inputs = keras.Input(shape=(163,))
d = keras.layers.Dense(32, activation='relu')(inputs)
d = keras.layers.Dropout(0.25)(d)
outputs = keras.layers.Dense(1, activation='relu')(d)


model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 163)]             0         
                                                                 
 dense_4 (Dense)             (None, 32)                5248      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5,281
Trainable params: 5,281
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(
    loss=keras.losses.MSE,
    optimizer=keras.optimizers.SGD(),
    metrics=['mae']
)

In [20]:
# model performance before training
model.evaluate(x_test,y_test)

837/837 [==============================] - 2s 2ms/step - loss: 0.0332 - mae: 0.1575


[0.03323846310377121, 0.15752136707305908]

In [23]:
history = model.fit(x_train, y_train, batch_size=16, epochs=10, validation_split=0.2)

Epoch 1/10
5356/5356 [==============================] - 17s 3ms/step - loss: 0.0048 - mae: 0.0498 - val_loss: 0.0017 - val_mae: 0.0339
Epoch 2/10
5356/5356 [==============================] - 20s 4ms/step - loss: 0.0018 - mae: 0.0340 - val_loss: 0.0015 - val_mae: 0.0311
Epoch 3/10
5356/5356 [==============================] - 16s 3ms/step - loss: 0.0016 - mae: 0.0318 - val_loss: 0.0013 - val_mae: 0.0293
Epoch 4/10
5356/5356 [==============================] - 15s 3ms/step - loss: 0.0014 - mae: 0.0305 - val_loss: 0.0012 - val_mae: 0.0280
Epoch 5/10
5356/5356 [==============================] - 15s 3ms/step - loss: 0.0014 - mae: 0.0295 - val_loss: 0.0011 - val_mae: 0.0269
Epoch 6/10
5356/5356 [==============================] - 16s 3ms/step - loss: 0.0013 - mae: 0.0286 - val_loss: 0.0010 - val_mae: 0.0259
Epoch 7/10
5356/5356 [==============================] - 15s 3ms/step - loss: 0.0012 - mae: 0.0278 - val_loss: 9.8807e-04 - val_mae: 0.0251
Epoch 8/10
5356/5356 [=============================

In [24]:
# model performance after training
model.evaluate(x_test,y_test)

837/837 [==============================] - 3s 3ms/step - loss: 8.7404e-04 - mae: 0.0233


[0.0008740443736314774, 0.0232991985976696]

Multi Task Neural Network

In [27]:
y_raw = y.copy()

for i in range(y_raw.shape[1]):
  print(f'task {i}:   min:{y_raw[:,i].min()}, max{y_raw[:,i].max()}')

task 0:   min:0.0, max29.5564
task 1:   min:6.31, max196.62
task 2:   min:-0.4286, max-0.1017
task 3:   min:-0.175, max0.1935
task 4:   min:0.0246, max0.6221
task 5:   min:19.0002, max3374.7532
task 6:   min:0.015951, max0.273944
task 7:   min:6.002, max46.969
task 8:   min:-714.568061, max-40.47893
task 9:   min:-714.560153, max-40.476062
task 10:   min:-714.559209, max-40.475117
task 11:   min:-714.602138, max-40.498597


some values are negative, final activation must be linear

experiment 1: multitask without label scaling

In [78]:
np.random.seed(42)
tf.random.set_seed(42)

In [79]:
x_raw = x.copy()
# train test split
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=0.20, random_state=42)

In [80]:
x_train.shape

(107108, 163)

In [81]:
y_train.shape

(107108, 12)

In [82]:
# model

inputs = keras.Input(shape=(163,))
d = keras.layers.Dense(32, activation='relu')(inputs)
d = keras.layers.Dropout(0.25)(d)
outputs = keras.layers.Dense(12, activation='linear')(d)


model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 163)]             0         
                                                                 
 dense_10 (Dense)            (None, 32)                5248      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_11 (Dense)            (None, 12)                396       
                                                                 
Total params: 5,644
Trainable params: 5,644
Non-trainable params: 0
_________________________________________________________________


In [83]:
model.compile(
    loss=keras.losses.MSE,
    optimizer=keras.optimizers.SGD(),
    metrics=['mae']
)

In [85]:
# model performance before training
_, pre_mae = model.evaluate(x_test,y_test)

837/837 [==============================] - 2s 2ms/step - loss: 182050.7500 - mae: 245.4659


In [87]:
history = model.fit(x_train, y_train, batch_size=16, epochs=10, validation_split=0.2)

Epoch 1/10
5356/5356 [==============================] - 18s 3ms/step - loss: 18322.3398 - mae: 47.1497 - val_loss: 6980.4438 - val_mae: 27.9728
Epoch 2/10
5356/5356 [==============================] - 16s 3ms/step - loss: 7059.2583 - mae: 28.0816 - val_loss: 6980.0532 - val_mae: 27.9870
Epoch 3/10
5356/5356 [==============================] - 16s 3ms/step - loss: 7059.0308 - mae: 28.0845 - val_loss: 6980.0991 - val_mae: 27.9815
Epoch 4/10
5356/5356 [==============================] - 17s 3ms/step - loss: 7059.1904 - mae: 28.0779 - val_loss: 6979.8594 - val_mae: 28.0180
Epoch 5/10
5356/5356 [==============================] - 18s 3ms/step - loss: 7058.7520 - mae: 28.0888 - val_loss: 6980.1714 - val_mae: 27.9777
Epoch 6/10
5356/5356 [==============================] - 15s 3ms/step - loss: 7059.2188 - mae: 28.0784 - val_loss: 6979.8818 - val_mae: 28.0138
Epoch 7/10
5356/5356 [==============================] - 14s 3ms/step - loss: 7058.9751 - mae: 28.0860 - val_loss: 6980.0615 - val_mae: 27.983

In [92]:
# model performance after training
_, post_mae = model.evaluate(x_test,y_test)

837/837 [==============================] - 2s 2ms/step - loss: 7149.0059 - mae: 28.1898


In [93]:
(pre_mae - post_mae) / pre_mae

0.8851581098749531

experiment 2: multitask with label scaling

In [94]:
np.random.seed(42)
tf.random.set_seed(42)

In [95]:
y_raw = y.copy()
x_raw = x.copy()
# train test split
x_train, x_test, y_train, y_test = train_test_split(x_raw, y_raw, test_size=0.20, random_state=42)

In [96]:
scale_factor = y_train.max(axis=0) - y_train.min(axis=0)
scale_shift = y_train.min(axis=0)

In [97]:
y_train_scaled =  ((y_train - scale_shift) / scale_factor)

In [98]:
print(y_train_scaled.min())
print(y_train_scaled.max())

0.0
1.0


In [99]:
# scale test set with train scaling values
y_test_scaled =  ((y_test - scale_shift) / scale_factor)

In [100]:
print(y_test_scaled.min())
print(y_test_scaled.max())

-0.16486912953997138
1.028406625377702


In [101]:
# model

inputs = keras.Input(shape=(163,))
d = keras.layers.Dense(32, activation='relu')(inputs)
d = keras.layers.Dropout(0.25)(d)
outputs = keras.layers.Dense(12, activation='linear')(d)


model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 163)]             0         
                                                                 
 dense_12 (Dense)            (None, 32)                5248      
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                                 
 dense_13 (Dense)            (None, 12)                396       
                                                                 
Total params: 5,644
Trainable params: 5,644
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.compile(
    loss=keras.losses.MSE,
    optimizer=keras.optimizers.SGD(),
    metrics=['mae']
)

In [103]:
# model performance before training
_, pre_mae = model.evaluate(x_test,y_test_scaled)

837/837 [==============================] - 2s 2ms/step - loss: 0.3271 - mae: 0.4665


In [104]:
history = model.fit(x_train, y_train_scaled, batch_size=16, epochs=10, validation_split=0.2)

Epoch 1/10
5356/5356 [==============================] - 16s 3ms/step - loss: 0.0220 - mae: 0.1082 - val_loss: 0.0062 - val_mae: 0.0599
Epoch 2/10
5356/5356 [==============================] - 14s 3ms/step - loss: 0.0069 - mae: 0.0630 - val_loss: 0.0052 - val_mae: 0.0538
Epoch 3/10
5356/5356 [==============================] - 15s 3ms/step - loss: 0.0057 - mae: 0.0566 - val_loss: 0.0050 - val_mae: 0.0524
Epoch 4/10
5356/5356 [==============================] - 15s 3ms/step - loss: 0.0054 - mae: 0.0550 - val_loss: 0.0048 - val_mae: 0.0515
Epoch 5/10
5356/5356 [==============================] - 14s 3ms/step - loss: 0.0053 - mae: 0.0541 - val_loss: 0.0047 - val_mae: 0.0507
Epoch 6/10
5356/5356 [==============================] - 14s 3ms/step - loss: 0.0052 - mae: 0.0535 - val_loss: 0.0046 - val_mae: 0.0500
Epoch 7/10
5356/5356 [==============================] - 14s 3ms/step - loss: 0.0051 - mae: 0.0529 - val_loss: 0.0045 - val_mae: 0.0494
Epoch 8/10
5356/5356 [==============================] -

In [105]:
# model performance after training
_, post_mae = model.evaluate(x_test,y_test_scaled)

837/837 [==============================] - 2s 2ms/step - loss: 0.0042 - mae: 0.0481


In [106]:
(pre_mae - post_mae) / pre_mae

0.8969096530981241

Scaling labels in this configuration had a very minor positive effect